In [2]:
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
from geo_math import *
import pygame

def cube(mouse_near, mouse_far):
    vertices = (
        (1,-1,-1), (1,1,-1), (-1,1,-1), (-1,-1,-1),
        (1,-1,1), (1,1,1), (-1,-1,1), (-1,1,1)
    )
    edegs = (
        (0,1), (0,3), (0,4), (2,1), (2,3), (2,7),
        (6,3), (6,4), (6,7), (5,1), (5,4), (5,7),
    )
    faces = (
        (0, 1, 2, 3),
        (3, 2, 7, 6),
        (6, 7, 5, 4),
        (4, 5, 1, 0),
        (4, 0, 3, 6),
        (1, 5, 7, 2),   
        
    )
    
    glEnable(GL_POLYGON_OFFSET_FILL)
    glPolygonOffset(1.0, 1.0)
    glBegin(GL_QUADS)
    glColor3fv((0.4,0.4,0.4))    
    for face in faces:
        if(isectQuad(mouse_near, mouse_far, vertices[face[0]], vertices[face[1]], vertices[face[2]], vertices[face[3]])):
            glColor3fv((1,0.4,0.4)) 
        else:
            glColor3fv((0.4,0.4,0.4))    
        for vertex in face:
            glVertex3fv(vertices[vertex])
    glEnd()

    glBegin(GL_LINES)
    glColor3fv((1,1,1))
    for edeg in edegs:
        for vertex in edeg:
            glVertex3fv(vertices[vertex])
    glEnd()


pygame.init()
pygame.display.set_caption('PyGame OpenGL')
pygame.display.set_icon(pygame.image.load('icon.png'))
screen = (800, 600)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

glEnable(GL_DEPTH_TEST)
glEnable(GL_LIGHTING)
glEnable(GL_LIGHT0)
glEnable(GL_COLOR_MATERIAL)

glMatrixMode(GL_PROJECTION)
gluPerspective(45, (screen[0]/screen[1]), 0.1, 50.0)

glMatrixMode(GL_MODELVIEW)  
glTranslate(0.0,0.0,-5)

rot_x, rot_y, zoom = 30, 45, -0.5
glClearColor(0, 0.4, 0.7, 1)

clock = pygame.time.Clock()
busy = True
mv_mat = (GLdouble * 16)()
p_mat  = (GLdouble * 16)()
v_rect = (GLint * 4)() 
while busy:
    try:
        mouse_buttons = pygame.mouse.get_pressed()
        button_down = mouse_buttons[0] == 1
    
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                busy = False
            elif event.type == pygame.MOUSEMOTION:
                if button_down:
                    rot_x = (rot_x + event.rel[1]) % 360
                    if rot_x > 90 and rot_x < 270:
                        rot_y = (rot_y - event.rel[0]) % 360
                    else:
                        rot_y = (rot_y + event.rel[0]) % 360
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4:
                    zoom += 0.2
                if event.button == 5:
                    zoom -= 0.2

        
        
        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        glPushMatrix()
        glLight(GL_LIGHT0, GL_POSITION,  (0, 1, 0, 1))
        glLightfv(GL_LIGHT0, GL_AMBIENT,  (1, 1, 1, 0))
        glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
        glTranslatef(0.0,0.0, zoom)
        glRotatef(rot_x, 1, 0, 0)  
        glRotatef(rot_y, 0, 1, 0)  
        glGetDoublev(GL_MODELVIEW_MATRIX, mv_mat)
        glGetDoublev(GL_PROJECTION_MATRIX, p_mat)
        glGetIntegerv(GL_VIEWPORT, v_rect)
        mouse_pos = pygame.mouse.get_pos()
        mouse_pos = mouse_pos[0], v_rect[3] - mouse_pos[1]


        temp_val = [GLdouble() for _ in range(3)]
        OpenGL.raw.GLU.gluUnProject(*mouse_pos, 0, mv_mat, p_mat, v_rect, *temp_val)
        mouse_near = [v.value for v in temp_val]    
        OpenGL.raw.GLU.gluUnProject(*mouse_pos, 1, mv_mat, p_mat, v_rect, *temp_val)
        mouse_far = [v.value for v in temp_val]

        cube(mouse_near, mouse_far)
        glPopMatrix()
        
        pygame.display.flip()
        clock.tick(100)
    except Exception as e:
        pygame.quit()
        raise e
        
pygame.quit()